# Как работать с nc файлами?

## Чтение из файла:
**nc** - это формат файла для хранения многомерных метеорологических величин. Форма массива может зависеть от широты, долготы, высоты и  времени.  

**netCDF4** -  специальный модуль для работы с файлами nc типа.

In [28]:
# из модуля импортируем Dataset
from netCDF4 import Dataset

# создаем объект класса Dataset, в качестве аргумента передаем имя файла, открываем в режиме (mode) чтения ('r')
# можно и не указывать режим при чтении, т.к. по умолчания он уже установлен.
nc = Dataset('Uzon_month_1958_2019_jra.nc', mode = 'r')

# теперь nc это объект/контейнер в котором содержится информация о многомерном массиве, 
# в частности о его размерности, название координатных осей, форма массива и т.д.

# Информацию о размерностях массива можно посмотреть в словаре dimensions
print(nc.dimensions)

{'lon': <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 1, 'lat': <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 36, 'lev': <class 'netCDF4._netCDF4.Dimension'>: name = 'lev', size = 17, 'time': <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 744}


In [36]:
# для того чтобы создать переменную, которой будет передана информация о размерности массива 
# нужно знать ее имя. Например 'lat' это имя координат широт, а размер - 36 точек.
# создадим с помощью numpy array переменную lat

#импортируем numpy
import numpy as np

# передаем широтные координаты в lat
lat = np.array(nc['lat'])

# lev - высотные уровни
lev = np.array(nc['lev']),

# временная шкала
time = np.array(nc['time'])
print('Широты =', lat,'\n\n',
      'Высоты =', lev)

Широты = [-87.5 -82.5 -77.5 -72.5 -67.5 -62.5 -57.5 -52.5 -47.5 -42.5 -37.5 -32.5
 -27.5 -22.5 -17.5 -12.5  -7.5  -2.5   2.5   7.5  12.5  17.5  22.5  27.5
  32.5  37.5  42.5  47.5  52.5  57.5  62.5  67.5  72.5  77.5  82.5  87.5] 

 Высоты = (array([ 1.421,  4.263,  7.105,  9.947, 12.789, 15.631, 18.473, 21.315,
       24.157, 26.999, 29.841, 32.683, 35.525, 38.367, 41.209, 44.051,
       46.893]),)


In [37]:
# подробную информацию в какой форме хранится информация в файле можно получить в словаре variables
# nc.variables

# здесь кроме информации о размерностях массива, можно узнать имя метеорологической величины,
# имя записывается в кавычках 'uzon', uzon(time, lev, lat, lon) - в скобках указана очередность
# размерностей массива, current shape = (744, 17, 36, 1).

# переменная data это массив среднезональной скорости ветра. 
data = np.array(nc['uzon'])

# файлы не всегда хранят одинаковую информацию, но можно подготовить
# блок, который можно изменять под конкретный 

'''
from netCDF4 import Dataset
import numpy as np
nc = Dataset('name.nc')
data = np.array(nc['uzon'])
lats = np.array(nc['lat'])
levs = np.array(nc['lev'])
time = np.array(nc['time'])
'''
nc.close()

## Запись в файл:

https://unidata.github.io/python-training/workshop/Bonus/netcdf-writing/
ссылка на семинар

In [49]:
from netCDF4 import Dataset
import numpy as np

# создаем объект(контейнер) с помощью Dataset в который нужно передать три параметра (аргумента) - 
# имя файла, режим и формат. 'w' - в данном режиме, если существует файл с похожим именем, 
# то хранящиеся данные будут стерты и запишутся новые данные.

ncfile = Dataset('new.nc', mode = 'w', format = 'NETCDF4_CLASSIC')

# ncfile это пустой контейнер, в который постепенно мы будем добавлять информация.
#### создание аттрибутов(аттрибуты - дополнительная информация, записывается по желанию создателя файла)

ncfile.title = 'My data'
ncfile.subtitle = 'Monthly mean'
ncfile.author = 'RSHU'

#Представьте что ncfile это пустая тетрадь -'new.nc'
#Эту тетрадь мы будем использовать только для записи информации в нее (mode = 'w').
#Подпишем эту тетрадь: укажем имя, для чего она предназначена- это и есть аттрибуты 

print(ncfile)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: My data
    subtitle: Monthly mean
    author: RSHU
    dimensions(sizes): 
    variables(dimensions): 
    groups: 


In [50]:
#### создание осей - dimensions(sizes)
# укажем форму наших осей массива с помощью метода createDimension
# У каждой оси есть имя(key) и длина(value). Обычно имя переменной это короткое обозначение. 
# эта информация записывается далее в словарь ncfile.dimensions

lat_dim = ncfile.createDimension('lat', 73)   
lon_dim = ncfile.createDimension('lon', 144)    
time_dim = ncfile.createDimension('time', 100) 

print(len(lat_dim))

# можно получить доступ к ключам и значениям словаря через
# ncfile.dimensions.items() - к ключам и значениям
# ncfile.dimensions.keys() - к ключам
# ncfile.dimensions.value() - к значениям

# for dim in ncfile.dimensions.items():
#     print(dim)

# Наша тетрадь все еще пустая, но мы уже запланировали что на каждой странице мы будем записывать часовые значения
# и таких страниц будет 100. И еще на каждой странице мы будем рисовать таблицу состоящую 
# из 73 строк и 144 стоблцов. 

73


In [51]:
#### создание переменных - variables(dimensions):
# каждой переменной соответсвует ее имя, тип данных, форма (форма определеяется как кортеж из названия осей)
# переменные имеют аттрибуты, такие units(величина измерения), long_name (полное имя переменной)

# метод createVariables принимает три обязательных аргумента:
# - 1ый - название переменной (строка)
# - 2ой - тип данных
# - 3ий - кортеж, с названием оси (который должен быть уже создан)


lat = ncfile.createVariable('lat', np.float32, ('lat',))
lat.units = 'degrees_north'
lat.long_name = 'latitude'

lon = ncfile.createVariable('lon', np.float32, ('lon',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'

time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'hours since 1800-01-01'
time.long_name = 'time'

# Объявляем 3х мерную переменную в которой будут записаны данные о температуре
temp = ncfile.createVariable('temp',np.float64,('lat','lon', 'time')) 
temp.units = 'K' 
temp.standard_name = 'air_temperature' 

# Запишем где нибудь на краю страницы, более детальную информацию про размеры таблицы. 
# Назовем строки - 'lat', положению каждой строки будет соответствовать
# какое то числовое значение так что 32 бита будет достаточно для его записи, указываем 
# запланированное количества строк через через имя оси ('lat',). 

# Теперь представим, что эта таблица представляет поверхность земли разбитую на квадраты. а строки и столбцы 
# это  широты и долготы.
# На каждой странице на пересечений линий будут находится метеонаблюдатели, которые будут сообщают вам 
# значение температуры воздуха у поверхности земли каждый час. Но в каких координатах мы пока не знаем, 
# потому что еще не определили расстояние между широтами и долготами. 
# Опять же на краю страницы вы записываете эти детали.
# На этом этапе во подготовили каркас, в котором пока неизвестно расстояние между линиями, и данные о температуре.


In [47]:
# после создания переменных в них нужно поместить информацию 
# определяем длину осей
nlats = len(lat_dim)
nlons = len(lon_dim)
ntimes = len(time_dim)

# заполняем переменные lat, lon, time значениями
lat[:] = -90. + (180./nlats)*np.arange(nlats) # от южного к северному
lon[:] = (180./nlons)*np.arange(nlons) # Greenwich meridian eastward
time[:] = np.arange(ntimes)

# создадим 3хмерный массив случайных чисел
data_arr = np.random.uniform(low=280,high=330,size=(nlats,nlons,ntimes))

# запишем данные в переменную temp
temp[:,:,:] = data_arr 

# закрыть файл
ncfile.close()
print('Dataset is closed!')

# Здесь мы установили шаг между строками и столбцами, метеоналюдатели находятся в 
# определенных координатах по широте и долготе и сообщают вам измеряемые значения. А вы все это записываете в тетрадь
# Чтобы получить доступ к данным температуры вам нужно обратиться в переменной temp.

# Дальше будет только интереснее и легче!

Dataset is closed!


In [17]:

'''
ncfile = Dataset('new.nc', mode = 'w', format = 'NETCDF4_CLASSIC')

# размерности осей
lat_dim = ncfile.createDimension('lat', 73)   
lon_dim = ncfile.createDimension('lon', 144)    
time_dim = ncfile.createDimension('time', 100) 

# cоздание переменных
lat = ncfile.createVariable('lat', np.float32, ('lat',))
lat.units = 'degrees_north'
lat.long_name = 'latitude'

lon = ncfile.createVariable('lon', np.float32, ('lon',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'

time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'hours since 1800-01-01'
time.long_name = 'time'

temp = ncfile.createVariable('temp',np.float64,('lat','lon', 'time')) 
temp.units = 'K' 
temp.standard_name = 'air_temperature' 

# запись данных
nlats = len(lat_dim); nlons = len(lon_dim); ntimes = len(time_dim)
lat[:] = -90. + (180./nlats)*np.arange(nlats) # от южного к северному
lon[:] = (180./nlons)*np.arange(nlons) # Greenwich meridian eastward
time[:] = np.arange(ntimes)
# temp[:,:,:] = массив данных

# закрыть файл
ncfile.close(); print('Dataset is closed!')
'''

"\nncfile = Dataset('new.nc', mode = 'w', format = 'NETCDF4_CLASSIC')\n\n# размерности осей\nlat_dim = ncfile.createDimension('lat', 73)   \nlon_dim = ncfile.createDimension('lon', 144)    \ntime_dim = ncfile.createDimension('time', 100) \n\n# cоздание переменных\nlat = ncfile.createVariable('lat', np.float32, ('lat',))\nlat.units = 'degrees_north'\nlat.long_name = 'latitude'\n\nlon = ncfile.createVariable('lon', np.float32, ('lon',))\nlon.units = 'degrees_east'\nlon.long_name = 'longitude'\n\ntime = ncfile.createVariable('time', np.float64, ('time',))\ntime.units = 'hours since 1800-01-01'\ntime.long_name = 'time'\n\ntemp = ncfile.createVariable('temp',np.float64,('lat','lon', 'time')) \ntemp.units = 'K' \ntemp.standard_name = 'air_temperature' \n\n# запись данных\nnlats = len(lat_dim); nlons = len(lon_dim); ntimes = len(time_dim)\nlat[:] = -90. + (180./nlats)*np.arange(nlats) # от южного к северному\nlon[:] = (180./nlons)*np.arange(nlons) # Greenwich meridian eastward\ntime[:] = np.a